In [7]:
from environment import *

### Grab URL for all weather sites

In [6]:
URL = 'https://reg.bom.gov.au/vic/observations/vicall.shtml'

def weather_api():
    response = requests.get(URL)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        content_div = soup.findAll("th", class_ = "rowleftcolumn")
        apis = dict()
        for row in content_div:
            if "href" in str(row):
                url_match = re.search(r'href="([^"]+)', str(row))
                name_match = re.search(r'id="([^"]+)', str(row))
                api = "https://reg.bom.gov.au/" + url_match.group(1).replace('products', 'fwo').replace('shtml', 'json')
                apis[name_match.group(1)] = api
        with open("../data/weather_APIs.json", 'w') as f:
            json.dump(apis, f)

In [ ]:
weather_api()

### Match Site location with SA2 area

In [ ]:
SITE_SA2 = []
def find_sa2(lon, lat, sf):
    point = Point(lon, lat)
    for _, row in sf.iterrows():
        sa2_code = row['SA2_CODE21']
        sa2_name = row['SA2_NAME21']
        sa2_site = row['geometry']
        if point.within(sa2_site):
            return sa2_code, sa2_name
    return "not found", "not found"

def site_sa2():
    sf = gpd.read_file("../SA2_shape/SA2_2021_AUST_GDA2020.shp")
    sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    with open("weather_APIs.json", 'r') as f:
        weather_apis = json.load(f)
    for site, url in weather_apis.items():
        try:
            response = requests.get(url, timeout=60)
            if response.status_code == 200:
                data = response.json()
                lon, lat = data["observations"]["data"][0].get("lon", ""), data["observations"]["data"][0].get("lat", "")
                sa2_code, sa2_name = find_sa2(lon, lat, sf)
                SITE_SA2.append({"site": site, "sa2_code": sa2_code, "sa2_name": sa2_name, "location": (lon, lat)})
        except requests.exceptions.Timeout:
            print()
        except requests.exceptions.RequestException as e:
            print(e)
            
    return None



In [ ]:
site_sa2()
with open("../data/site_sa2.json", 'w') as f:
    json.dump(SITE_SA2, f)

### Find centroid for all sa2 area

In [10]:
sf = gpd.read_file("../SA2_shape/SA2_2021_AUST_GDA2020.shp")
sf = sf[["SA2_CODE21", "SA2_NAME21", "geometry"]]
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf = sf.dropna()

In [13]:
all_sa2 = []
for i in range(len(sf)):
    row = sf.iloc[i]
    point = row['geometry'].centroid
    all_sa2.append({
        "sa2_code": row["SA2_CODE21"],
        "sa2_name": row["SA2_NAME21"],
        "lon": point.x,
        "lat": point.y
    })
with open("../data/All_SA2.json",'w') as f:
    json.dump(all_sa2, f)